# What do the models and their interaction graphs look like?

In [1]:
import mpbn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
modelname = []
modelname.extend([f"models/bn{n}.bnet" for n in range(1000)])

solutions = list()
for i,n in enumerate(modelname):
    solutions.append(mpbn.MPBooleanNetwork(f"{n}"))

`clauses_per_model` : list of models, each model being a dictionary
* keys = **model's nodes** 
* value = a set (-> **the clauses constituting the node's function in the model**)

In [3]:
# For each model, I store the clauses per node
clauses_per_model = list() #Each element of this list will correspond to a model, it's a dict linking nodes and their clauses.
for i, model in enumerate(solutions): #We go through the 1000 models.
    clauses_of_a_node = dict() #The dict that will link a node to its clauses.
    for node in model.keys(): #We go through the nodes of the model.
        #if not isinstance(model[node],frozenset):
        if model[node]==True or model[node]==False :
            clauses_of_a_node[node] = model[node]
        else:
            clauses_of_a_node[node] = mpbn.minibn.struct_of_dnf(model.ba, model[node]) #We store the clauses ruling this node.
    clauses_per_model.append(clauses_of_a_node) #Now that all nodes are linked with their clauses, we save this information as a new element in clauses_per_model.

`influences` : dictionnary
* keys = **model's nodes** 
* value = dictionary  
     + keys = **its inhibitors and activators**
     + value = **number of models having this inhibitor/activator in the node's function**

`constantes` : dictionnary
* keys = **model's nodes with at least one constant function in a model (constant function : 1 or 0)**
* value = dictionnary
     + key = **value of the constant function (TRUE for 1, FALSE for 0)**
     + value = **number of models having 1 or 0 as function for the node**

In [25]:
influences = dict()
constantes = dict()
for model in clauses_per_model:
    for node, influenceurs_set in model.items():
        temp = set()
        if node not in influences:
            influences[node] = dict()
        #print(influenceurs_set)
        #print(type(influenceurs_set))
        #print(isinstance(influenceurs_set, bool))
        if influenceurs_set != True and influenceurs_set != False:
            for conjonctions in influenceurs_set:
                for clause in conjonctions:
                    if clause[0] not in temp:
                        temp.add(clause[0])
                        if clause[0] not in influences[node]:
                            influences[node][clause[0]] = dict()
                            influences[node][clause[0]][clause[1]] = 1
                        elif clause[1] not in influences[node][clause[0]]:
                            influences[node][clause[0]][clause[1]] = 1
                        else:
                            influences[node][clause[0]][clause[1]] += 1
        else:
            if node not in constantes:
                constantes[node] = dict()
            if influenceurs_set not in constantes[node]:
                constantes[node][influenceurs_set] = 1
            else:
                constantes[node][influenceurs_set] += 1

In [5]:
# The number of models with this influencer in the function of the node:
for node,influenceurs in influences.items():
    print(f"-- {node} --")
    for influenceur, influence in influenceurs.items():
        print(f"{influenceur}: {influence}")

-- ATF3 --
JUN: {True: 1000}
CREM: {True: 889}
TRP53: {True: 885}
KLF6: {True: 885}
-- ATF7 --
MYC: {True: 1000}
-- CEBPA --
HNF4A: {True: 1000}
MYC: {False: 995}
MYB: {True: 110}
-- CREM --
FOS: {True: 995}
SP1: {True: 995}
STAT1: {True: 995}
JUN: {True: 995}
-- E2F1 --
TRP53: {False: 1000}
ESR1: {True: 112}
SP1: {True: 110}
MYC: {True: 110}
-- EBF1 --
TBX21: {True: 1000}
-- ESR1 --
STAT1: {True: 1000}
STAT3: {True: 995}
TRP53: {True: 995}
SP1: {True: 995}
FOXO3: {True: 995}
-- FLI1 --
EBF1: {True: 1000}
MEF2C: {True: 942}
SPI1: {True: 890}
-- FOS --
JUN: {True: 999}
STAT3: {True: 997}
ESR1: {True: 970}
STAT6: {True: 891}
SP1: {True: 996}
STAT1: {True: 995}
FLI1: {True: 996}
CREM: {False: 995}
TRP53: {True: 995}
SPI1: {True: 961}
-- FOXO3 --
-- FOXP2 --
-- GATA1 --
FLI1: {True: 1000}
-- GFI1B --
GATA1: {True: 1000}
-- HBP1 --
FOXO3: {True: 1000}
MYC: {True: 110}
-- HNF4A --
SP1: {True: 997}
CEBPA: {True: 998}
TRP53: {False: 886}
-- IKZF1 --
TBX21: {True: 1000}
ZEB2: {False: 1}
-- IRF1

**To know the number of possible functions for each node:**

In [7]:
fonctions_differentes = dict()
fonctions_differentes_avec_occurrence = dict()

for node in influences.keys():
    fonctions_differentes[node] = set()
    fonctions_differentes_avec_occurrence[node] = dict()
    for model in clauses_per_model:
        fonctions_differentes[node].add(model[node])
        if model[node] not in fonctions_differentes_avec_occurrence[node]:
            fonctions_differentes_avec_occurrence[node][model[node]] = 1
        else:
            fonctions_differentes_avec_occurrence[node][model[node]] += 1

In [8]:
noeuds_communs_a_tout_modele = list()
for node, functions in fonctions_differentes.items():
    print(f"{node}: {len(functions)}")
    if len(functions) == 1:
        noeuds_communs_a_tout_modele.append(node)

ATF3: 3
ATF7: 1
CEBPA: 3
CREM: 4
E2F1: 3
EBF1: 1
ESR1: 2
FLI1: 3
FOS: 682
FOXO3: 1
FOXP2: 1
GATA1: 1
GFI1B: 1
HBP1: 2
HNF4A: 5
IKZF1: 2
IRF1: 5
IRF2: 4
JUN: 5
JUNB: 4
KLF1: 1
KLF6: 2
MEF2C: 2
MYB: 5
MYC: 16
NFATC1: 2
NRF1: 1
RELA: 1
RUNX1: 6
SP1: 2
SPI1: 3
STAT1: 4
STAT3: 5
STAT6: 1
TBP: 1
TBX21: 2
TCF12: 2
TRP53: 44
ZEB2: 1


In [10]:
print(f"Nodes common to all models :\n{noeuds_communs_a_tout_modele}")

Nodes common to all models :
['ATF7', 'EBF1', 'FOXO3', 'FOXP2', 'GATA1', 'GFI1B', 'KLF1', 'NRF1', 'RELA', 'STAT6', 'TBP', 'ZEB2']


**To know, for each possible function, the number of models having it:**

In [11]:
def nb_of_models_having_each_function(node):
    for fonction, occurrence in fonctions_differentes_avec_occurrence[node].items():
        print(f"-- {occurrence:3d}:")
        for clause in fonction:
            print(f"{clause}")

## Creation of the influence graph, with:

* on the arcs, the number of models (among the 1000) that have the influence
* dark blue highlighting of nodes with a constant function in at least one model
* on each node, the number of different functions possible on the 1000 models

In [14]:
import graphviz

In [15]:
from typing import List, Dict


def get_arcs_size(l_input: List, min_intensity: int = 1, max_intensity: int = 10) -> Dict[int, str]:
    res = {}
    get_arc_size_rec(l_input, res, min_intensity, max_intensity)
    return res
       
def get_arc_size_rec(l_rec: List, res: Dict[int, str], inf: int, sup: int, left: bool = False):
    max_diff = 0
    index_max_diff = 0
    for i in range(len(l_rec) - 1):
        diff = l_rec[i + 1] - l_rec[i]
        if diff > max_diff:
            max_diff = diff
            index_max_diff = i

    if max_diff == 0 or inf == sup:
        for element in l_rec:
            if left:
                res[element] = str(inf)
            else:
                res[element] = str(sup)
    else:
        get_arc_size_rec(l_rec[0:index_max_diff+1], res, inf, (inf+sup)//2, True)
        get_arc_size_rec(l_rec[index_max_diff+1:], res, (inf+sup)//2 + 1, sup)

### Analysis of common edges and nodes in the models:

In [17]:
occurrences = dict()
for target, sources in influences.items():
    for source, influences in sources.items():
        for influence, occurrence in influences.items():
            if occurrence not in occurrences:
                occurrences[occurrence] = list()
            occurrences[occurrence].append((source,target))

max_intensity = 8
intensity = get_arcs_size(sorted(occurrences.keys()), max_intensity=max_intensity)
intensity[1000]=str(max_intensity+5)

**To know the number of arcs present in X models:**

In [18]:
for occurrence, arcs in sorted(occurrences.items()):
    print(f"{len(arcs):2d} arcs présents dans {occurrence:3d} modèles")

 1 arcs présents dans   1 modèles
 2 arcs présents dans   3 modèles
 1 arcs présents dans  41 modèles
 1 arcs présents dans  57 modèles
 1 arcs présents dans  94 modèles
 6 arcs présents dans 110 modèles
 1 arcs présents dans 112 modèles
 5 arcs présents dans 885 modèles
 2 arcs présents dans 886 modèles
 2 arcs présents dans 889 modèles
 2 arcs présents dans 890 modèles
 2 arcs présents dans 891 modèles
 1 arcs présents dans 906 modèles
 1 arcs présents dans 942 modèles
 1 arcs présents dans 961 modèles
 1 arcs présents dans 970 modèles
 1 arcs présents dans 974 modèles
22 arcs présents dans 995 modèles
14 arcs présents dans 996 modèles
11 arcs présents dans 997 modèles
 9 arcs présents dans 998 modèles
 4 arcs présents dans 999 modèles
33 arcs présents dans 1000 modèles


### Creation of knots, colored and shaped according to their variability through the models

* bright yellow if identical function in all models
* pale yellow if 2 possible functions
* beige if 3 possible functions

In [19]:
models_per_edge = graphviz.Digraph('models_per_edge', comment='Display the number of models having each edge in its influence graph.')
models_per_edge.graph_attr['ratio'] = '1.1'

In [20]:
# Creation of non-constant nodes:

for node in fonctions_differentes:
    if node not in constantes:
        if len(fonctions_differentes[node]) == 1:
            models_per_edge.node(node, label=f"{node}\n{len(fonctions_differentes[node])} func", fillcolor="darkgoldenrod2", style="rounded,filled,bold", shape="oval", fontcolor="black", fontname="arial bold", fontsize="50pt")
        elif len(fonctions_differentes[node]) == 2:
            models_per_edge.node(node, label=f"{node}\n{len(fonctions_differentes[node])} func", fillcolor="lightgoldenrod1" , style="rounded,filled", shape="oval", fontsize="40pt")
        elif len(fonctions_differentes[node]) == 3:
            models_per_edge.node(node, label=f"{node}\n{len(fonctions_differentes[node])} func", fillcolor="cornsilk" , style="rounded,filled", shape="oval", fontsize="35pt")
        elif len(fonctions_differentes[node]) < 10:
            models_per_edge.node(node, label=f"{node}\n{len(fonctions_differentes[node])} func", fillcolor="white" , style="rounded,filled", shape="oval", fontsize="30pt")
        else:
            models_per_edge.node(node, label=f"{node}\n{len(fonctions_differentes[node])} func", fillcolor="white", style="rounded,filled,dotted", shape="oval", fontsize="30pt")

In [21]:
# Creation of constant nodes:

for constante, influence in constantes.items():
    for signe, occurrence in influence.items():
        #signe = 1 if signe == True else 0
        if len(fonctions_differentes[constante]) == 1:
            models_per_edge.node(constante, label=f"{constante}\n{len(fonctions_differentes[constante])} func\n{signe}: {occurrence}", fillcolor="darkgoldenrod2", style="rounded,filled,bold", shape="box", fontcolor="black", fontname="arial bold", fontsize="45pt")
        elif len(fonctions_differentes[constante]) == 2:
            models_per_edge.node(constante, label=f"{constante}\n{len(fonctions_differentes[constante])} func\n{signe}: {occurrence}", fillcolor="lightgoldenrod1", style="rounded,filled", shape="box", fontsize="35pt")
        elif len(fonctions_differentes[constante]) == 3:
            models_per_edge.node(constante, label=f"{constante}\n{len(fonctions_differentes[constante])} func\n{signe}: {occurrence}", fillcolor="cornsilk", style="rounded,filled", shape="box", fontsize="30pt")
        elif len(fonctions_differentes[constante]) < 10:
            models_per_edge.node(constante, label=f"{constante}\n{len(fonctions_differentes[constante])} func\n{signe}: {occurrence}", fillcolor="white", style="rounded,filled", shape="box", fontsize="25pt")
        else:
            models_per_edge.node(constante, label=f"{constante}\n{len(fonctions_differentes[constante])} func\n{signe}: {occurrence}", fillcolor="white", style="rounded,filled,dotted", shape="box", fontsize="25pt")

In [26]:
# Creation of the arcs, colored according to the activating/inhibiting influence:

occurrences_liste = list()
for target, sources in influences.items():
    #if target=="STAT3":
    for source, infl in sources.items():
        #print(f"{source}, {infl}")
        for influence, occurrence in infl.items():
            occurrences_liste.append(occurrence)
            if influence == True:
                if (source, target) in occurrences[1000]:
                    models_per_edge.edge(source, target, label=f"{occurrence}", penwidth=intensity[occurrence], color="darkgreen", fontcolor="darkgreen", fontname="arial bold", fontsize="30pt", arrowsize="2")
                else:
                    # Ligne commentée correspond au graphe sans utilisation de l'intensité pour définir la largeur de la flèche.
                    #models_per_edge.edge(source, target, label=str(occurrence), penwidth=str(occurrence/100), color="green")
                    models_per_edge.edge(source, target, label=str(occurrence), penwidth=intensity[occurrence], color="darkolivegreen3", fontcolor="darkolivegreen3", fontname="arial bold", fontsize="30pt", arrowsize="2")
            else:
                if (source, target) in occurrences[1000]:
                    models_per_edge.edge(source, target, label=str(occurrence), arrowhead="tee", penwidth=intensity[occurrence], color="firebrick", fontcolor="firebrick", fontname="arial bold", fontsize="30pt", arrowsize="3")
                else:
                    #models_per_edge.edge(source, target, label=str(occurrence), arrowhead="tee", penwidth=str(occurrence/100), color="red"
                    models_per_edge.edge(source, target, label=str(occurrence), arrowhead="tee", penwidth=intensity[occurrence], color="lightcoral", fontcolor="lightcoral", fontname="arial bold", fontsize="30pt", arrowsize="3")
            #print(f"{source} {target} {influence} {occurrence}")

**Saving the graph:**

In [28]:
# Save to a PDF file named "models_per_edge.gv.pdf"
models_per_edge.render(filename="IGstat.pdf", directory='data/analysis/', view=True)

'data/analysis/IGstat.pdf.pdf'